In [2]:
import gradio as gr
import os
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_mistralai.chat_models import ChatMistralAI

# Load environment variables from .env file
load_dotenv()

# Get Mistral API key from environment variables
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

# Game state to track conversation history
class GameState:
    def __init__(self):
        self.messages = []
        self.game_type = None
        self.game_started = False

# Initialize game state
game_state = GameState()

# System prompts for different game types
SYSTEM_PROMPTS = {
    "sci-fi": """You are the game master for a text-based sci-fi adventure game. 
    Create an immersive sci-fi adventure set in a distant future where the player is a space explorer.
    Guide the player through an interactive story with challenges, decisions, and consequences.
    Describe the environment vividly and respond to player actions naturally.
    The player can type any action or response they want, and you should interpret their intent and continue the story accordingly.
    Suggest possible actions the player might consider, but make it clear they can respond however they want.
    Keep the story engaging and adjust based on player decisions.
    Begin by introducing the setting and the player's initial situation.""",
    
    "fantasy": """You are the game master for a text-based fantasy adventure game.
    Create an immersive fantasy adventure set in a world of magic, dragons, and ancient kingdoms.
    Guide the player through an interactive story with challenges, decisions, and consequences.
    Describe the environment vividly and respond to player actions naturally.
    The player can type any action or response they want, and you should interpret their intent and continue the story accordingly.
    Suggest possible actions the player might consider, but make it clear they can respond however they want.
    Keep the story engaging and adjust based on player decisions.
    Begin by introducing the setting and the player's initial situation."""
}

# Initialize the Mistral AI chat model
mistral = ChatMistralAI(
    model="mistral-small",
    temperature=0.7,
    max_tokens=500,
    mistral_api_key=MISTRAL_API_KEY
)

def get_mistral_response(messages):
    """Get a response from Mistral API using LangChain"""
    try:
        response = mistral.invoke(messages)
        return response.content
    except Exception as e:
        return f"Error communicating with Mistral API: {str(e)}"

def chat(message, history):
    """Process user messages and maintain game state"""
    global game_state

    # First interaction - game type selection
    if not game_state.game_type:
        if "sci-fi" in message.lower():
            game_state.game_type = "sci-fi"
            game_state.game_started = True
            
            # Initialize with system prompt
            system_message = SystemMessage(content=SYSTEM_PROMPTS["sci-fi"])
            user_message = HumanMessage(content="Start the sci-fi adventure.")
            
            game_state.messages = [system_message, user_message]
            
            response = get_mistral_response(game_state.messages)
            return [(message, response)]
            
        elif "fantasy" in message.lower():
            game_state.game_type = "fantasy"
            game_state.game_started = True
            
            # Initialize with system prompt
            system_message = SystemMessage(content=SYSTEM_PROMPTS["fantasy"])
            user_message = HumanMessage(content="Start the fantasy adventure.")
            
            game_state.messages = [system_message, user_message]
            
            response = get_mistral_response(game_state.messages)
            return [(message, response)]
            
        else:
            return [(message, "Please select a game type: Sci-Fi or Fantasy")]
    
    # Continue existing game
    else:
        # Add user message to history
        game_state.messages.append(HumanMessage(content=message))
        
        # Get response
        response = get_mistral_response(game_state.messages)
        
        # Update the displayed chat history
        updated_history = history + [(message, response)]
        return updated_history

def reset_game():
    """Reset the game state"""
    global game_state
    game_state = GameState()
    return [("System", "Game reset. Please select a game type: Sci-Fi or Fantasy")]

# Create Gradio interface
with gr.Blocks(title="Text Adventure Game") as demo:
    gr.Markdown("# Text Adventure Game")
    gr.Markdown("### Choose your adventure type: Sci-Fi or Fantasy")
    gr.Markdown("#### After selecting a game type, you can type any action or response you want!")
    
    chatbot = gr.Chatbot(height=500)
    msg = gr.Textbox(placeholder="Enter your choice or action...", label="Your Input")
    clear = gr.Button("Start New Game")
    
    msg.submit(fn=chat, inputs=[msg, chatbot], outputs=chatbot)
    clear.click(fn=reset_game, inputs=None, outputs=chatbot)
    
    # Initialize with welcome message
    chatbot.value = [("System", "Welcome to the Text Adventure Game! Please select a game type: Sci-Fi or Fantasy")]

# Launch the app
if __name__ == "__main__":
    demo.launch()

C:\Users\mnoch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\mnoch\AppData\Local\Temp\ipykernel_46420\3184943567.py:119: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500)


* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


In [1]:
pip install langchain-mistralai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\mnoch\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
